In [ ]:
from geometry_msgs.msg import PoseStamped
from pycram.graph_of_convex_sets import BoundingBox, GraphOfConvexSets
from pycram.process_module import simulated_robot
from pycram.world_concepts.world_object import Object
from pycram.datastructures.enums import WorldMode, GripperState, Arms, WaypointsMovementType
from pycrap.ontologies import PhysicalObject, Robot, Kitchen
from pycram.object_descriptors.urdf import ObjectDescription
from pycram.datastructures.pose import Pose
from pycram.worlds.bullet_world import BulletWorld
from pycram.object_descriptors.generic import ObjectDescription as GenericObjectDescription
from pycram.ros_utils.viz_marker_publisher import VizMarkerPublisher
from pycram.ros_utils.robot_state_updater import WorldStateUpdater
from tf.transformations import quaternion_from_matrix
from pycram.designators.action_designator import *
from pycram.process_module import simulated_robot, real_robot
from pycram.external_interfaces.giskard import achieve_cartesian_waypoints_goal, sync_worlds, giskard_wrapper

from pycram.designators.motion_designator import MoveTCPWaypointsMotion
from geometry_msgs.msg import Quaternion

world = BulletWorld(mode=WorldMode.GUI)

viz_marker_publisher = VizMarkerPublisher()

kitchen = Object("kitchen", Kitchen, "kitchen.urdf")

# extension = ObjectDescription.get_file_extension()
robot = Object("pr2", Robot, f"pr2.urdf", pose=Pose([0, 1, 0]))
WorldStateUpdater(tf_topic="/tf", joint_state_topic="/joint_states")
sync_worlds()

In [ ]:
from pycram.external_interfaces.giskard import giskard_wrapper
kitchen.set_joint_position("oven_area_area_right_drawer_joint", .45)
giskard_wrapper.set_seed_configuration({"oven_area_area_right_drawer_joint": .45})
pose = PoseStamped()
pose.header.frame_id = 'map'
pose.pose.position.y = 1
pose.pose.orientation.w = 1
giskard_wrapper.monitors.add_set_seed_odometry(pose)
giskard_wrapper.execute()

In [ ]:
from pycram.datastructures.enums import WorldMode, Arms
with real_robot:
    try:
        ParkArmsActionDescription([Arms.BOTH]).resolve().perform()
    except:
        pass

In [ ]:
search_space = BoundingBox(min_x=-0.0, max_x=1.5, min_y=0.1, max_y=2, min_z=0, max_z=1.45)
print('Calculating graph of convex sets...')
cg = GraphOfConvexSets.free_space_from_world(world, search_space=search_space)

In [ ]:
print('reading current pose of the gripper...')
start_pose = robot.links['l_gripper_tool_frame'].pose
print('searching for path to goal pose...')
start = Pose([start_pose.position.x, start_pose.position.y, start_pose.position.z])
# goal_position = Pose([1, 1.49, 1.0])
goal_position = Pose([1, 1.49, 0.7])
goal_orientation = quaternion_from_matrix([[0, -1, 0, 0],
                                           [1, 0, 0, 0],
                                           [0, 0, 1, 0],
                                           [0, 0, 0, 1]])
path = cg.path_from_to(start, goal_position)
path[-1].pose.orientation = Quaternion(*goal_orientation)
print('done')

In [ ]:
print('move along path to goal pose...')
from pycram.datastructures.enums import MovementType, WaypointsMovementType
with real_robot:
    MoveTCPWaypointsMotion(path[1:], Arms.RIGHT, movement_type=WaypointsMovementType.ENFORCE_ORIENTATION_FINAL_POINT, allow_gripper_collision=False).perform()

In [ ]:
import numpy as np
print('path refinement...')
print('Original Path:', path)
print(len(path))
def filter_path(path, distance=0.05):
    new_path = []
    p_start = start
    new_path.append(start)
    for p in path[:-1]:
        dist = np.linalg.norm([p.position_as_list()[0] - p_start.position_as_list()[0], p.position_as_list()[1] - p_start.position_as_list()[1], p.position_as_list()[2] - p_start.position_as_list()[2]])
        if dist > distance:
            new_path.append(p)
        p_start = p
    new_path.append(path[-1])
    return new_path

new_path = filter_path(path)
print('New Path:', new_path)
print(len(new_path))


In [ ]:
print('move along path to goal pose...')
with real_robot:
    MoveTCPWaypointsMotion(filter_path(path), Arms.RIGHT).perform()